This tutorial can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.3_tutorials/wallaroo-run-anywhere/pipeline-edge-publish/edge-arbitrary-python).

## Arbitrary Python Edge Deploy

This tutorial demonstrates how to use arbitrary python as a ML Model in Wallaroo and deploy the pipeline, then deploy that same pipeline to edge devices through an Open Container Initiative Registry, registered in the Wallaroo instance as the Edge Deployment Registry.

### Tutorial Goals

* In Wallaroo Ops:
  * **Arbitrary Python Tutorial Deploy Model in Wallaroo Upload and Deploy**: Deploys the `KMeans` model in an arbitrary Python package in Wallaroo Ops, and perform sample inferences.  The file `models/model-auto-conversion-BYOP-vgg16-clustering.zip` is provided so users can go right to testing deployment.
  * **Publish pipeline to Open Container Initiative Registry, registered in the Wallaroo Ops instance as the Edge Deployment Registry.**:  This will containerize the pipeline and copy the engine, python steps, model, and deployment configuration to the registry service.
* In a remote aka edge device:
  * **Deploy the Pipeline to an Edge Device**:  The pipeline will be deployed to an Edge device as a remote Wallaroo Inference Server through Docker.

### Arbitrary Python Script Requirements

The entry point of the arbitrary python model is any python script that **must** include the following.

* `class ImageClustering(Inference)`:  The default inference class.  This is used to perform the actual inferences.  Wallaroo uses the `_predict` method to receive the inference data and call the appropriate functions for the inference.
  * `def __init__`:  Used to initialize this class and load in any other classes or other required settings.
  * `def expected_model_types`: Used by Wallaroo to anticipate what model types are used by the script.
  * `def model(self, model)`: Defines the model used for the inference.  Accepts the model instance used in the inference.
    * `self._raise_error_if_model_is_wrong_type(model)`: Returns the error if the wrong model type is used.  This verifies that only the anticipated model type is used for the inference.
    * `self._model = model`: Sets the submitted model as the model for this class, provided `_raise_error_if_model_is_wrong_type` is not raised.
  * `def _predict(self, input_data: InferenceData)`:  This is the entry point for Wallaroo to perform the inference.  This will receive the inference data, then perform whatever steps and return a dictionary of numpy arrays.
* `class ImageClusteringBuilder(InferenceBuilder)`: Loads the model and prepares it for inferencing.
  * `def inference(self) -> ImageClustering`: Sets the inference class being used for the inferences.
  * `def create(self, config: CustomInferenceConfig) -> ImageClustering`: Creates an inference subclass, assigning the model and any attributes required for it to function.

All other methods used for the functioning of these classes are optional, as long as they meet the requirements listed above.

### References

* [Wallaroo SDK Essentials Guide: Model Uploads and Registrations: Arbitrary Python](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/wallaroo-sdk-model-arbitrary-python/)
* [Wallaroo SDK Essentials Guide: Pipeline Edge Publication](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-publication/)

* ## Prerequisites

* A deployed Wallaroo Ops instance.
* A location with Docker or Kubernetes with `helm` for Wallaroo Inference server deployments.
* The following Python libraries installed:
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame


## Tutorial Steps

### Import Libraries

The first step is to import the libraries we'll be using.  These are included by default in the Wallaroo instance's JupyterHub service.

In [1]:
import numpy as np
import pandas as pd
import json
import pyarrow as pa
import wallaroo

from wallaroo.pipeline   import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework
from wallaroo.object import EntityNotFoundError

import requests

### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more details on logging in through Wallaroo, see the [Wallaroo SDK Essentials Guide: Client Connection](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
wl = wallaroo.Client()

ERROR:root:Keycloak token refresh got error: 400 - {"error":"invalid_grant","error_description":"Invalid refresh token"}


### Set Variables

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.



In [3]:
workspace_name = f'vgg16-clustering-workspace'
pipeline_name = f'vgg16-clustering-pipeline'

model_name = 'vgg16-clustering'
model_file_name = './models/model-auto-conversion-BYOP-vgg16-clustering.zip'

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline that is used to deploy our arbitrary Python model.

In [4]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)
wl.set_current_workspace(workspace)

pipeline = wl.build_pipeline(pipeline_name)

### Upload Arbitrary Python Model

Arbitrary Python models are uploaded to Wallaroo through the Wallaroo Client `upload_model` method.

#### Upload Arbitrary Python Model Parameters

The following parameters are required for Arbitrary Python models.  Note that while some fields are considered as **optional** for the `upload_model` method, they are required for proper uploading of a Arbitrary Python model to Wallaroo.

| Parameter | Type | Description |
|---|---|---|
|`name` | `string` (*Required*) | The name of the model.  Model names are unique per workspace.  Models that are uploaded with the same name are assigned as a new **version** of the model. |
|`path` | `string` (*Required*) | The path to the model file being uploaded. |
|`framework` |`string` (*Upload Method Optional, Arbitrary Python model Required*) | Set as `Framework.CUSTOM`. |
|`input_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, Arbitrary Python model Required*) | The input schema in Apache Arrow schema format. |
|`output_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, Arbitrary Python model Required*) | The output schema in Apache Arrow schema format. |
| `convert_wait` | `bool` (*Upload Method Optional, Arbitrary Python model Optional*) (*Default: True*) | <ul><li>**True**: Waits in the script for the model conversion completion.</li><li>**False**:  Proceeds with the script without waiting for the model conversion process to display complete. |

Once the upload process starts, the model is containerized by the Wallaroo instance.  This process may take up to 10 minutes.

#### Upload Arbitrary Python Model Return

The following is returned with a successful model upload and conversion.

| Field | Type | Description |
|---|---|---|
| `name` | string | The name of the model. |
| `version` | string | The model version as a unique UUID. |
| `file_name` | string | The file name of the model as stored in Wallaroo. |
| `image_path` | string | The image used to deploy the model in the Wallaroo engine. |
| `last_update_time` | DateTime | When the model was last updated. |

For our example, we'll start with setting the `input_schema` and `output_schema` that is expected by our `ImageClustering._predict()` method.

In [5]:
input_schema = pa.schema([
    pa.field('images', pa.list_(
        pa.list_(
            pa.list_(
                pa.int64(),
                list_size=3
            ),
            list_size=32
        ),
        list_size=32
    )),
])

output_schema = pa.schema([
    pa.field('predictions', pa.int64()),
])

### Upload Model

Now we'll upload our model.  The framework is `Framework.CUSTOM` for arbitrary Python models, and we'll specify the input and output schemas for the upload.

In [19]:
model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=Framework.CUSTOM, 
                        input_schema=input_schema, 
                        output_schema=output_schema, 
                        convert_wait=False)

# time to finish the auto-packaging
import time
time.sleep(120)

Once the model is uploaded, we wait for the auto-packaging process to complete.  We retrieve the model with `wallaroo.client.Client.get_model` which accepts the model name, and returns the latest version of the model.

In [21]:
model = wl.get_model(model_name)
model

Name,vgg16-clustering
Version,1e557953-2fc7-4258-a851-3042adbe913a
File Name,model-auto-conversion-BYOP-vgg16-clustering.zip
SHA,7bb3362b1768c92ea7e593451b2b8913d3b7616c19fd8d25b73fb6990f9283e0
Status,ready
Image Path,proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mac-deploy:v2024.1.0-main-4963
Architecture,x86
Acceleration,none
Updated At,2024-22-Apr 16:41:43


### Deploy Pipeline

The model is uploaded and ready for use.  We'll add it as a step in our pipeline, then deploy the pipeline.  For this example we're allocated 0.25 cpu and 4 Gi RAM to the pipeline through the pipeline's deployment configuration.

In [22]:
# clear if used before
pipeline.clear()
pipeline.add_model_step(model)

name,vgg16-clustering-pipeline
created,2024-04-22 16:17:08.651909+00:00
last_updated,2024-04-22 16:38:12.074544+00:00
deployed,True
arch,x86
accel,none
tags,
versions,"412b8da5-ad4c-417c-9f6e-ad79d71522a4, 4233c4e7-517a-48e8-807a-b626834f45ec, 4ca1d45a-507d-42e2-8038-d608c543681a, a99f0a28-ad9e-4db3-9eea-113bdd9ca1cd, be19886c-3896-47d5-9935-35592f44ad7c"
steps,vgg16-clustering
published,False


In [23]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(1).memory('4Gi') \
    .build()

pipeline.deploy(deployment_config=deployment_config)

name,vgg16-clustering-pipeline
created,2024-04-22 16:17:08.651909+00:00
last_updated,2024-04-22 16:43:31.697272+00:00
deployed,True
arch,x86
accel,none
tags,
versions,"29d94f80-3c21-44fb-9e71-a5498c3bce3d, 412b8da5-ad4c-417c-9f6e-ad79d71522a4, 4233c4e7-517a-48e8-807a-b626834f45ec, 4ca1d45a-507d-42e2-8038-d608c543681a, a99f0a28-ad9e-4db3-9eea-113bdd9ca1cd, be19886c-3896-47d5-9935-35592f44ad7c"
steps,vgg16-clustering
published,False


### Run inference

Everything is in place - we'll now run a sample inference with some toy data.  In this case we're randomly generating some values in the data shape the model expects, then submitting an inference request through our deployed pipeline.

In [25]:
deploy_url = pipeline._deployment._url()

headers = wl.auth.auth_header()

headers['Content-Type']='application/json; format=pandas-records'
headers['Accept']='application/json; format=pandas-records'

dataFile = './data/vgg16_test.df.json'

In [26]:
# inference through Pipeline Deploy URL

!curl -X POST {deploy_url} \
     -H "Authorization:{headers['Authorization']}" \
     -H "Content-Type:{headers['Content-Type']}" \
     -H "Accept:{headers['Accept']}" \
     --data @{dataFile}

[{"time":1713804243509,"in":{"images":[23,109,44,60,212,137,45,255,159,203,92,99,75,244,173,169,213,186,247,81,161,135,206,44,150,191,1,73,60,210,61,198,189,3,27,29,225,138,32,184,1,88,137,36,207,149,212,226,114,173,4,68,226,177,254,75,105,51,29,168,119,230,221,128,1,135,104,246,46,164,69,120,21,98,143,173,113,167,100,71,128,119,162,98,191,108,102,103,180,246,155,13,9,214,67,7,81,189,166,182,5,229,242,143,42,237,179,142,116,51,0,205,150,204,43,173,219,249,105,105,234,196,132,45,195,248,208,135,154,254,5,244,48,169,227,9,164,72,92,36,203,162,165,155,64,77,242,110,14,44,238,197,203,159,190,205,9,6,247,34,58,38,171,237,228,15,244,79,147,97,81,199,251,244,89,156,154,96,125,45,91,26,147,40,186,91,252,213,113,13,168,130,61,217,10,252,115,167,163,66,246,98,24,120,46,255,249,56,37,65,222,206,185,39,54,154,26,123,93,95,38,34,197,24,149,212,120,170,87,11,1,29,84,244,101,117,51,53,196,8,201,127,232,117,128,63,138,60,59,65,25,153,160,52,21,251,186,63,143,36,196,128,136,206,31,249,122,70,167,245,78

### Undeploy Pipelines

The inference is successful, so we will undeploy the pipeline and return the resources back to the cluster.

In [27]:
pipeline.undeploy()

name,vgg16-clustering-pipeline
created,2024-04-22 16:17:08.651909+00:00
last_updated,2024-04-22 16:43:31.697272+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"29d94f80-3c21-44fb-9e71-a5498c3bce3d, 412b8da5-ad4c-417c-9f6e-ad79d71522a4, 4233c4e7-517a-48e8-807a-b626834f45ec, 4ca1d45a-507d-42e2-8038-d608c543681a, a99f0a28-ad9e-4db3-9eea-113bdd9ca1cd, be19886c-3896-47d5-9935-35592f44ad7c"
steps,vgg16-clustering
published,False


### Publish the Pipeline for Edge Deployment

It worked! For a demo, we'll take working once as "tested". So now that we've tested our pipeline, we are ready to publish it for edge deployment.

Publishing it means assembling all of the configuration files and model assets and pushing them to an Open Container Initiative (OCI) repository set in the Wallaroo instance as the Edge Registry service.  DevOps engineers then retrieve that image and deploy it through Docker, Kubernetes, or similar deployments.

See [Edge Deployment Registry Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/) for details on adding an OCI Registry Service to Wallaroo as the Edge Deployment Registry.

This is done through the SDK command `wallaroo.pipeline.publish(deployment_config)` which has the following parameters and returns.

#### Publish a Pipeline Parameters

The `publish` method takes the following parameters.  The containerized pipeline will be pushed to the Edge registry service with the model, pipeline configurations, and other artifacts needed to deploy the pipeline.

| Parameter | Type | Description |
|---|---|---|
| `deployment_config` | `wallaroo.deployment_config.DeploymentConfig` (*Optional*) | Sets the pipeline deployment configuration.  For example:    For more information on pipeline deployment configuration, see the [Wallaroo SDK Essentials Guide: Pipeline Deployment Configuration](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-deployment-config/).

#### Publish a Pipeline Returns

| Field | Type | Description |
|---|---|---|
| id | integer | Numerical Wallaroo id of the published pipeline. |
| pipeline version id | integer | Numerical Wallaroo id of the pipeline version published. |
| status | string | The status of the pipeline publication.  Values include:  <ul><li>PendingPublish: The pipeline publication is about to be uploaded or is in the process of being uploaded.</li><li>Published:  The pipeline is published and ready for use.</li></ul> |
| Engine URL | string | The URL of the published pipeline engine in the edge registry. |
| Pipeline URL | string | The URL of the published pipeline in the edge registry. |
| Helm Chart URL | string | The URL of the helm chart for the published pipeline in the edge registry. |
| Helm Chart Reference | string | The help chart reference. |
| Helm Chart Version | string | The version of the Helm Chart of the published pipeline.  This is also used as the Docker tag. |
| Engine Config | `wallaroo.deployment_config.DeploymentConfig` | The pipeline configuration included with the published pipeline. |
| Replaces | List | A list of any pipeline publishes replaced by this new publish. |
| Docker Run | string | Template of commands for deploying via `docker run`. |
| Helm Install | string | Template of commands for deploying via `helm`. |
| Created At | DateTime | When the published pipeline was created. |
| Updated At | DateTime | When the published pipeline was updated. |

### Publish Example

We will now publish the pipeline to our Edge Deployment Registry with the `pipeline.publish(deployment_config)` command.  `deployment_config` is an optional field that specifies the pipeline deployment.  This can be overridden by the DevOps engineer during deployment.

In [28]:
pub = pipeline.publish(deployment_config)
pub

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is publishing............... Published.


ID,4
Pipeline Name,vgg16-clustering-pipeline
Pipeline Version,7a7509d5-c30b-4f33-82ae-49deaf79dbd1
Status,Published
Engine URL,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4963
Pipeline URL,ghcr.io/wallaroolabs/doc-samples/pipelines/vgg16-clustering-pipeline:7a7509d5-c30b-4f33-82ae-49deaf79dbd1
Helm Chart URL,oci://ghcr.io/wallaroolabs/doc-samples/charts/vgg16-clustering-pipeline
Helm Chart Reference,ghcr.io/wallaroolabs/doc-samples/charts@sha256:0a68ac1fc42292263dad5e51431f59b859917357fdc2b8ab4c53f3e175180303
Helm Chart Version,0.0.1-7a7509d5-c30b-4f33-82ae-49deaf79dbd1
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 1.0, 'memory': '512Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': {}}}"
User Images,[]


### List Published Pipeline

The method `wallaroo.client.list_pipelines()` shows a list of all pipelines in the Wallaroo instance, and includes the `published` field that indicates whether the pipeline was published to the registry (`True`), or has not yet been published (`False`).

In [29]:
wl.list_pipelines()

[{'name': 'vgg16-clustering-pipeline', 'create_time': datetime.datetime(2024, 4, 22, 16, 17, 8, 651909, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'new-edge-inline-replacement', 'create_time': datetime.datetime(2024, 4, 22, 15, 43, 4, 574641, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'edge-inline-replacement-demon', 'create_time': datetime.datetime(2024, 4, 22, 15, 27, 36, 828826, tzinfo=tzutc()), 'definition': '[]'}]

### List Publishes from a Pipeline

All publishes created from a pipeline are displayed with the `wallaroo.pipeline.publishes` method.  The `pipeline_version_id` is used to know what version of the pipeline was used in that specific publish.  This allows for pipelines to be updated over time, and newer versions to be sent and tracked to the Edge Deployment Registry service.

#### List Publishes Parameters

N/A

#### List Publishes Returns

A List of the following fields:

| Field | Type | Description |
|---|---|---|
| id | integer | Numerical Wallaroo id of the published pipeline. |
| pipeline_version_id | integer | Numerical Wallaroo id of the pipeline version published. |
| engine_url | string | The URL of the published pipeline engine in the edge registry. |
| pipeline_url | string | The URL of the published pipeline in the edge registry. |
| created_by | string | The email address of the user that published the pipeline.
| Created At | DateTime | When the published pipeline was created. |
| Updated At | DateTime | When the published pipeline was updated. |

In [30]:
pipeline.publishes()

id,pipeline_version_name,engine_url,pipeline_url,created_by,created_at,updated_at
4,7a7509d5-c30b-4f33-82ae-49deaf79dbd1,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4963,ghcr.io/wallaroolabs/doc-samples/pipelines/vgg16-clustering-pipeline:7a7509d5-c30b-4f33-82ae-49deaf79dbd1,john.hummel@wallaroo.ai,2024-22-Apr 16:48:03,2024-22-Apr 16:48:03


## DevOps Deployment

We now have our pipeline published to our Edge Registry service.  We can deploy this in a x86 environment running Docker that is logged into the same registry service that we deployed to.

For more details, check with the documentation on your artifact service.  The following are provided for the three major cloud services:

* [Set up authentication for Docker](https://cloud.google.com/artifact-registry/docs/docker/authentication)
* [Authenticate with an Azure container registry](https://learn.microsoft.com/en-us/azure/container-registry/container-registry-authentication?tabs=azure-cli)
* [Authenticating Amazon ECR Repositories for Docker CLI with Credential Helper](https://aws.amazon.com/blogs/compute/authenticating-amazon-ecr-repositories-for-docker-cli-with-credential-helper/)

Once a pipeline is deployed to the Edge Registry service, it can be deployed in environments such as Docker, Kubernetes, or similar container running services by a DevOps engineer.  For full details, see [How to Publish and Deploy AI Workloads for For Edge/Multicloud Model Deployments](https://docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-run-anywhere/wallaroo-model-operations-run-anywhere-deploy/).  The pipeline publishes `Docker Run Command` and `Helm Install Command` provide templates for deployment.

## Edge Deployed Pipeline API Endpoints

Once deployed, we can check the pipelines and models available.  We'll use a `curl` command, but any HTTP based request will work the same way.

The endpoint `/pipelines` returns:

* **id** (*String*):  The name of the pipeline.
* **status** (*String*):  The status as either `Running`, or `Error` if there are any issues.

For this example, the deployment is made on a machine called `testboy.local`.  Replace this URL with the URL of you edge deployment.


In [38]:
!curl testboy.local:8080/pipelines

{"pipelines":[{"id":"vgg16-clustering-pipeline","version":"7a7509d5-c30b-4f33-82ae-49deaf79dbd1","status":"Running"}]}

The endpoint `/models` returns a List of models with the following fields:

* **name** (*String*): The model name.
* **sha** (*String*): The sha hash value of the ML model.
* **status** (*String*):  The status of either Running or Error if there are any issues.
* **version** (*String*):  The model version.  This matches the version designation used by Wallaroo to track model versions in UUID format.

In [39]:
!curl testboy.local:8080/models

{"models":[{"sha":"7bb3362b1768c92ea7e593451b2b8913d3b7616c19fd8d25b73fb6990f9283e0","name":"vgg16-clustering","version":"1e557953-2fc7-4258-a851-3042adbe913a","status":"Running"}]}

### Edge Inference Endpoint

The inference endpoint takes the following pattern:

* `/infer`:  The inference endpoint remains the same regardless of the pipeline or models deployed.  This allows publish replacements without altering the inference endpoint used by other applications.

Wallaroo inference endpoint URLs accept the following data inputs through the `Content-Type` header:

* `Content-Type: application/vnd.apache.arrow.file`: For Apache Arrow tables.
* `Content-Type: application/json; format=pandas-records`: For pandas DataFrame in record format.

Once deployed, we can perform an inference through the deployment URL.

The endpoint returns `Content-Type: application/json; format=pandas-records` by default with the following fields:

* **check_failures** (*List[Integer]*): Whether any validation checks were triggered.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Management: Anomaly Testing](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/#anomaly-testing).
* **elapsed** (*List[Integer]*): A list of time in nanoseconds for:
  * [0] The time to serialize the input.
  * [1...n] How long each step took.
* **model_name** (*String*): The name of the model used.
* **model_version** (*String*): The version of the model in UUID format.
* **original_data**: The original input data.  Returns `null` if the input may be too long for a proper return.
* **outputs** (*List*): The outputs of the inference result separated by data type, where each data type includes:
  * **data**: The returned values.
  * **dim** (*List[Integer]*): The dimension shape returned.
  * **v** (*Integer*): The vector shape of the data.
* **pipeline_name**  (*String*): The name of the pipeline.
* **shadow_data**: Any shadow deployed data inferences in the same format as **outputs**.
* **time** (*Integer*): The time since UNIX epoch.

In [ ]:
# set the content type and accept headers
headers = {
    'Content-Type': 'application/json; format=pandas-records'
}

dataFile = './data/vgg16_test.df.json'

!curl -X POST 'http://testboy.local:8080/infer' \
     -H "Content-Type:{headers['Content-Type']}" \
     --data @{dataFile}